In [1]:
import os
import pickle
import glob
import random
import time

import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats as st
# from sklearn import ensemble, svm
# from sklearn.metrics import balanced_accuracy_score, classification_report
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import RobustScaler, StandardScaler

import npc_lims
from npc_sessions import DynamicRoutingSession
# import npc_sessions.utils
from dynamic_routing_analysis import spike_utils, decoding_utils


ModuleNotFoundError: No module named 'charset_normalizer.assets'

In [ ]:
all_ephys_sessions = tuple(s for s in npc_lims.get_session_info(is_ephys=True, is_uploaded=True, is_annotated=True))

In [ ]:
session_projects={
    'session':[],
    'project':[]
}
for ss in all_ephys_sessions:
    session_projects['session'].append(str(ss.id+''))
    session_projects['project'].append(str(ss.project))
# session_list=list(decoder_results.keys())
# for ss in session_list:
#     session=DynamicRoutingSession(ss)
#     session_projects['session'].append(session.project)

session_projects=pd.DataFrame(session_projects)
session_projects

In [ ]:
# session.

In [ ]:
# npc_lims.get_subject_data_assets(649943)

In [ ]:
# all_ephys_sessions

In [3]:
# loadpath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\new_annotations\linear_shift_facemap_face_use_more_trials_500_svds"
loadpath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\test_same_5_folds"

In [4]:
#loop through decoder results, append to dict
# files=glob.glob(os.path.join(loadpath,'*_decoding_results*.pkl'))
files=glob.glob(os.path.join(loadpath,'*test_same_5_folds.pkl'))

decoder_results={}
for ii,ff in enumerate(files):
    with open(ff,'rb') as f:
        data=pickle.load(f)
    # session_id=ff.split('\\')[-1].split('_decoding')[0]
    session_id=ff.split('\\')[-1].split('_test')[0]
    decoder_results[session_id]=data[list(data.keys())[0]]


c:\Anaconda3\envs\npc_analysis\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.4.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.4.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.4.0 when using version 1.3.1. This might lead to br

In [ ]:
# #load templeton units
# loadpath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\linear_shift\combined\Templ_decoder_results.pkl"
# with open(loadpath,'rb') as f:
#     decoder_results=pickle.load(f)

# #load DR units
# loadpath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\linear_shift\combined\DR_decoder_results.pkl"
# with open(loadpath,'rb') as f:
#     decoder_results=pickle.load(f)

In [ ]:
# sessions=list(decoder_results.keys())
# sessions

In [ ]:
# decoder_results['620263_2022-07-26_0'].keys()

In [ ]:
# decoder_results[sessions[0]].keys()
# decoder_results[session_id]['results'][aa]['shift'].keys()
shifts[half_shift_inds]

In [ ]:
##TODO:
#add option to not discount half of the shifts ... how to do automatically?

In [ ]:
list(decoder_results[session_id]['results'][aa]['shift'].keys())

In [ ]:
decoder_results[session_id].keys()

In [51]:
use_half_shifts=False

session_id=list(decoder_results.keys())[0]

shifts=decoder_results[session_id]['shifts']
areas=decoder_results[session_id]['areas']


half_neg_shift=np.round(shifts.min()/2)
half_pos_shift=np.round(shifts.max()/2)

# half_shifts=np.arange(-half_neg_shift,half_pos_shift+1)
half_neg_shift_ind=np.where(shifts==half_neg_shift)[0][0]
half_pos_shift_ind=np.where(shifts==half_pos_shift)[0][0]
half_shift_inds=np.arange(half_neg_shift_ind,half_pos_shift_ind+1)

n_repeats=25
n_units=['all']

nu=n_units[0]

bal_acc={}
for aa in areas:
    if aa in decoder_results[session_id]['results']:
        bal_acc[aa]={}
        if use_half_shifts==False:
            half_shift_inds=decoder_results[session_id]['shifts']
        # for nu in n_units:
        for rr in range(n_repeats):
            if nu=='all' and rr>0:
                continue
            if rr in decoder_results[session_id]['results'][aa]['shift'][nu].keys():
                bal_acc[aa][rr]=[]
                for sh in half_shift_inds:
                    if sh in list(decoder_results[session_id]['results'][aa]['shift'][nu][rr].keys()):
                        bal_acc[aa][rr].append(decoder_results[session_id]['results'][aa]['shift'][nu][rr][sh]['balanced_accuracy_test'])
                # bal_acc[aa][rr]=np.array(bal_acc[aa][rr])

        # bal_acc[aa]=np.nanmean(np.vstack(bal_acc[aa]))

for aa in areas:
    if aa in decoder_results[session_id]['results']:
        true_acc=bal_acc[aa][shifts[half_shift_inds]==1]
        pval=np.round(np.mean(bal_acc[aa]>=true_acc),decimals=4)
        
        fig,ax=plt.subplots(1,1)
        ax.axhline(true_acc,color='k',linestyle='--',alpha=0.5)
        ax.axvline(1,color='k',linestyle='--',alpha=0.5)
        # ax.plot(shifts,bal_acc[aa])
        ax.plot(shifts[half_shift_inds],bal_acc[aa])
        ax.set_xlabel('trial shift')
        ax.set_ylabel('balanced accuracy')
        # ax.set_title(aa+' n='+str(decoder_results[session_id]['results'][aa]['n_units'])+' p='+str(pval))
        ax.set_title(str(aa)+' p='+str(pval))


TypeError: unhashable type: 'numpy.ndarray'

In [52]:
# decoder_results[session_id]['results'][aa]['shift']['all'][0][0]['balanced_accuracy_test']
# decoder_results[session_id]['shifts']
# decoder_results[session_id]['results'][aa]['shift'][nu][0][sh]['balanced_accuracy_test']
# half_shift_inds
bal_acc

{'ACAd': {0: [0.6752112486996209,
   0.6515926708949964,
   0.6707406288801637,
   0.6518571092989698,
   0.6524292761502063,
   0.6463688378804658,
   0.6590885600187926,
   0.6404812242021544,
   0.6681405416289137,
   0.6939586898889225,
   0.6705772005772006,
   0.6984294774992449,
   0.6930806402899427,
   0.677478942246384,
   0.6686053223262526,
   0.6701998389207692,
   0.6700645994832042,
   0.6863854491761469,
   0.6800360750360751,
   0.7261928252625927,
   0.6633189033189033,
   0.7026067988858686,
   0.6910293969596295,
   0.6915609584214236,
   0.6935212591026545,
   0.6869648310345984,
   0.73242877277761,
   0.6958924796134098,
   0.6926695526695527,
   0.7491961139635557,
   0.6773853149434546,
   0.7268708681499378,
   0.7230586596865666,
   0.748320581227558,
   0.7580225846504917,
   0.7373973958857679,
   0.7512550756736803,
   0.7696483103459848,
   0.7605141112117856,
   0.7552662841034934,
   0.7876156918017384,
   0.7538403973287694,
   0.8046504916272358,
   0

In [ ]:
#plot all sessions on same axis, plus average
sel_project='DynamicRouting'
use_half_shifts=False
session_list=list(decoder_results.keys())
bal_acc={}
norm_acc={}
all_shifts={}
# for session_id in sessions:
    # if session_id not in session_list:
    #     continue
for session_id in session_list:
    str_session_id=str(session_id)
    session_info=npc_lims.get_session_info(session_id)
    if session_info.is_annotated==False:
        continue
    if sel_project not in session_projects.query('session==@str_session_id')['project'].values[0]:
        continue
    shifts=decoder_results[session_id]['shifts']
    areas=decoder_results[session_id]['areas']

    half_neg_shift=np.round(shifts.min()/2)
    half_pos_shift=np.round(shifts.max()/2)
    # half_shifts=np.arange(-half_neg_shift,half_pos_shift+1)
    half_neg_shift_ind=np.where(shifts==half_neg_shift)[0][0]
    half_pos_shift_ind=np.where(shifts==half_pos_shift)[0][0]
    half_shift_inds=np.arange(half_neg_shift_ind,half_pos_shift_ind+1)

    for aa in areas:
        if aa in decoder_results[session_id]['results']:
            if use_half_shifts==False:
                half_shift_inds=np.asarray(list(decoder_results[session_id]['results'][aa]['shift'].keys()))
            temp_bal_acc=[]
            if type(aa)==str and '_probe' in aa:
                area_name=aa.split('_probe')[0]
            else:
                area_name=aa
            if area_name not in bal_acc:
                bal_acc[area_name]=[]
                norm_acc[area_name]=[]
                all_shifts[area_name]=[]
            for sh in half_shift_inds:
                temp_bal_acc.append(decoder_results[session_id]['results'][aa]['shift'][sh]['balanced_accuracy'])
            temp_bal_acc=np.array(temp_bal_acc)
            bal_acc[area_name].append(temp_bal_acc)
            true_acc=temp_bal_acc[shifts[half_shift_inds]==1]
            norm_acc[area_name].append(temp_bal_acc-true_acc)
            all_shifts[area_name].append(shifts[half_shift_inds])


In [ ]:
# decoder_results[list(decoder_results.keys())[0]]

In [ ]:
# sessions

In [ ]:
# bal_acc['MOs']

In [ ]:
areas=list(bal_acc.keys())
for ia, aa in enumerate(areas):
    if type(aa)==str and '_probe' in aa:
        areas[ia]=aa.split('_probe')[0]
areas

In [ ]:
for aa in areas:
    if aa not in bal_acc:
        continue
    # aa='all'
    xvect=np.arange(-100,101)
    stacked_shift_results=np.full((len(bal_acc[aa]),len(xvect)),np.nan)
    norm_stacked_shift_results=np.full((len(bal_acc[aa]),len(xvect)),np.nan)
    for xx in range(len(bal_acc[aa])):
        shift_start=np.where(xvect==all_shifts[aa][xx][0])[0][0]
        shift_end=np.where(xvect==all_shifts[aa][xx][-1])[0][0]+1
        stacked_shift_results[xx,shift_start:shift_end]=bal_acc[aa][xx]
        norm_stacked_shift_results[xx,shift_start:shift_end]=norm_acc[aa][xx]
        # ax.plot(all_shifts[aa][xx],norm_acc[aa][xx],color='k',alpha=0.1)

    mean_shift_results=np.copy(stacked_shift_results)
    mean_shift_results[:,np.where((xvect<-40)|(xvect>40))[0]]=np.nan

    mean_norm_shift_results=np.copy(norm_stacked_shift_results)
    mean_norm_shift_results[:,np.where((xvect<-40)|(xvect>40))[0]]=np.nan

    fig,ax=plt.subplots(2,1,figsize=(5,6))
    ax[0].axvline(1,color='k',linestyle='--',alpha=0.5)
    ax[0].plot(xvect,stacked_shift_results.T,color='k',alpha=0.1)
    ax[0].plot(xvect,np.nanmean(mean_shift_results,axis=0),color='k',linewidth=2)
    ax[0].set_ylabel('balanced accuracy')
    ax[1].axvline(1,color='k',linestyle='--',alpha=0.5)
    ax[1].plot(xvect,norm_stacked_shift_results.T,color='k',alpha=0.1)
    ax[1].plot(xvect,np.nanmean(mean_norm_shift_results,axis=0),color='k',linewidth=2)
    ax[1].set_ylabel('normalized balanced accuracy')
    ax[1].set_xlabel('trial shift')
    # fig.suptitle(sel_project+' sessions')
    fig.suptitle(aa+'('+str(len(bal_acc[aa]))+'); '+sel_project)


In [ ]:
# decoder_results['626791_2022-08-16'].keys()
# session_info=npc_lims.get_session_info(session_id)
# performance=pd.read_parquet(
#             npc_lims.get_cache_path('performance',session_info.id,version='0.0.214')
#             )
performance['cross_modal_dprime'].mean()

In [ ]:
#add ability to deal with _probe appended to area
#keep track of which probe each area row is from -- can I get this from the decoder_results dict for non-duplicated areas?

In [ ]:
session_list=list(decoder_results.keys())
use_half_shifts=False
has_repeats=True

all_bal_acc={}
linear_shift_dict={
    'session_id':[],
    'project':[],
    'area':[],
    'true_accuracy':[],
    'null_accuracy_mean':[],
    'null_accuracy_median':[],
    'null_accuracy_std':[],
    'p_value':[],
    'ccf_ap_mean':[],
    'ccf_dv_mean':[],
    'ccf_ml_mean':[],
    'n_units':[],
    'probe':[],
    'cross_modal_dprime':[],
    'n_good_blocks':[],
}

#loop through sessions
for session_id in session_list:
    session_info=npc_lims.get_session_info(session_id)
    try:
        performance=pd.read_parquet(
                    npc_lims.get_cache_path('performance',session_info.id,version='any')
                )
    except:
        continue

    if session_info.is_annotated==False:
        continue

    all_bal_acc[session_id]={}


    shifts=decoder_results[session_id]['shifts']
    #extract results according to the trial shift
    half_neg_shift=np.round(shifts.min()/2)
    half_pos_shift=np.round(shifts.max()/2)
    # half_shifts=np.arange(-half_neg_shift,half_pos_shift+1)
    half_neg_shift_ind=np.where(shifts==half_neg_shift)[0][0]
    half_pos_shift_ind=np.where(shifts==half_pos_shift)[0][0]
    half_shift_inds=np.arange(half_neg_shift_ind,half_pos_shift_ind+1)

    all_bal_acc[session_id]['shifts']=shifts
    all_bal_acc[session_id]['half_shift_inds']=half_shift_inds
    if use_half_shifts:
        half_shifts=shifts[half_shift_inds]
    else:
        half_shifts=shifts
    
    areas=decoder_results[session_id]['areas']

    #save balanced accuracy by shift
    for aa in areas:
        if aa in decoder_results[session_id]['results']:
            if use_half_shifts==False:
                half_shift_inds=np.asarray(list(decoder_results[session_id]['results'][aa]['shift'].keys()))
                half_shifts=shifts[half_shift_inds]
            all_bal_acc[session_id][aa]=[]
            for sh in half_shift_inds:
                all_bal_acc[session_id][aa].append(decoder_results[session_id]['results'][aa]['shift'][sh]['balanced_accuracy'])
            all_bal_acc[session_id][aa]=np.array(all_bal_acc[session_id][aa])

            if type(aa)==str:
                if '_probe' in aa:
                    area_name=aa.split('_probe')[0]
                    probe_name=aa.split('_probe')[1]
                else:
                    area_name=aa
                    probe_name=''
            else:
                area_name=aa
            
            true_acc_ind=np.where(half_shifts==1)[0][0]
            null_acc_ind=np.where(half_shifts!=1)[0]
            true_accuracy=all_bal_acc[session_id][aa][true_acc_ind]
            null_accuracy_mean=np.mean(all_bal_acc[session_id][aa][null_acc_ind])
            null_accuracy_median=np.median(all_bal_acc[session_id][aa][null_acc_ind])
            null_accuracy_std=np.std(all_bal_acc[session_id][aa][null_acc_ind])
            p_value=np.mean(all_bal_acc[session_id][aa][null_acc_ind]>=true_accuracy)

            #make big dict/dataframe for this:
            #save true decoding, mean/median null decoding, and p value for each area/probe
            linear_shift_dict['session_id'].append(session_id)
            linear_shift_dict['project'].append(session_projects.query('session==@session_id')['project'].values[0])
            linear_shift_dict['area'].append(area_name)
            linear_shift_dict['true_accuracy'].append(true_accuracy)
            linear_shift_dict['null_accuracy_mean'].append(null_accuracy_mean)
            linear_shift_dict['null_accuracy_median'].append(null_accuracy_median)
            linear_shift_dict['null_accuracy_std'].append(null_accuracy_std)
            linear_shift_dict['p_value'].append(p_value)

            linear_shift_dict['cross_modal_dprime'].append(performance['cross_modal_dprime'].mean())
            linear_shift_dict['n_good_blocks'].append(np.sum(performance['cross_modal_dprime']>=1.0))

            # 'ccf_ap_mean', 'ccf_dv_mean', 'ccf_ml_mean'
            if 'ccf_ap_mean' in decoder_results[session_id]['results'][aa].keys():
                linear_shift_dict['ccf_ap_mean'].append(decoder_results[session_id]['results'][aa]['ccf_ap_mean'])
                linear_shift_dict['ccf_dv_mean'].append(decoder_results[session_id]['results'][aa]['ccf_dv_mean'])
                linear_shift_dict['ccf_ml_mean'].append(decoder_results[session_id]['results'][aa]['ccf_ml_mean'])
                linear_shift_dict['n_units'].append(decoder_results[session_id]['results'][aa]['n_units'])
                linear_shift_dict['probe'].append(probe_name)
            else:
                linear_shift_dict['ccf_ap_mean'].append(np.nan)
                linear_shift_dict['ccf_dv_mean'].append(np.nan)
                linear_shift_dict['ccf_ml_mean'].append(np.nan)
                linear_shift_dict['n_units'].append(np.nan)
                linear_shift_dict['probe'].append(np.nan)

    
linear_shift_df=pd.DataFrame(linear_shift_dict)
linear_shift_df.to_csv(os.path.join(loadpath,'combined','all_unit_linear_shift_use_more_trials.csv'))

In [ ]:
# linear_shift_df.to_csv(os.path.join(loadpath,'combined','all_unit_linear_shift_use_more_trials.csv'))

In [ ]:
# linear_shift_df.to_csv(os.path.join(loadpath,'combined','all_unit_linear_shift_no_cues_results.csv'))

In [ ]:
# session_projects.query('session==@session_id')['project'].values
# session_info=npc_lims.get_session_info(session_id)
# session_info.is_annotated
linear_shift_df=pd.read_csv(r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\new_annotations\linear_shift_facemap_face_use_more_trials_500_svds\combined\all_unit_linear_shift_use_more_trials.csv")

In [ ]:
linear_shift_df

In [ ]:
#plot null median vs. n_units

#get median by bins
n_bins=15
n_units_bins=np.linspace(20,400,n_bins)
binned_n_units=np.full(n_bins-1,np.nan)
for ii in range(n_bins-1):
    binned_n_units[ii]=np.nanmedian(linear_shift_df.query('area!="all" & n_units>=@n_units_bins[@ii] & n_units<@n_units_bins[@ii+1]')['null_accuracy_median'].values)

#get pearson correlation
r,p=st.pearsonr(linear_shift_df.query('area!="all"')['n_units'],linear_shift_df.query('area!="all"')['null_accuracy_median'])

fig,ax=plt.subplots(1,1)
ax.plot(n_units_bins[:-1],binned_n_units,color='k',linewidth=2.0)
ax.scatter(linear_shift_df.query('area!="all"')['n_units'],linear_shift_df.query('area!="all"')['null_accuracy_median'],s=5,alpha=0.2)
ax.set_xlabel('n_units')
ax.set_ylabel('null_accuracy_median')
ax.set_xlim([0,350])
ax.set_title('r='+str(np.round(r,2))+' p='+str(np.round(p,8)))

In [ ]:
#plot null median vs. n_units using "all" units only

#get median by bins
n_bins=8
n_units_bins=np.linspace(500,3000,n_bins)
binned_n_units=np.full(n_bins-1,np.nan)
for ii in range(n_bins-1):
    binned_n_units[ii]=np.nanmedian(linear_shift_df.query('area=="all" & n_units>=@n_units_bins[@ii] & n_units<@n_units_bins[@ii+1]')['null_accuracy_median'].values)

#get pearson correlation
r,p=st.pearsonr(linear_shift_df.query('area=="all"')['n_units'],linear_shift_df.query('area=="all"')['null_accuracy_median'])

fig,ax=plt.subplots(1,1)
ax.plot(n_units_bins[:-1],binned_n_units,color='k',linewidth=2.0)
ax.scatter(linear_shift_df.query('area=="all"')['n_units'],linear_shift_df.query('area=="all"')['null_accuracy_median'],s=5,alpha=0.5)
ax.set_xlabel('n_units')

ax.set_ylabel('null_accuracy_median')
ax.set_xlim([0,3000])
ax.set_title('r='+str(np.round(r,2))+' p='+str(np.round(p,8)))


In [ ]:
#plot null median vs. n_units using "MOs" units only

#get median by bins
n_bins=10
n_units_bins=np.linspace(20,360,n_bins)
binned_n_units=np.full(n_bins-1,np.nan)
for ii in range(n_bins-1):
    binned_n_units[ii]=np.nanmedian(linear_shift_df.query('area=="MOs" & n_units>=@n_units_bins[@ii] & n_units<@n_units_bins[@ii+1]')['null_accuracy_median'].values)

#get pearson correlation
r,p=st.pearsonr(linear_shift_df.query('area=="MOs"')['n_units'],linear_shift_df.query('area=="MOs"')['null_accuracy_median'])

fig,ax=plt.subplots(1,1)
ax.plot(n_units_bins[:-1],binned_n_units,color='k',linewidth=2.0)
ax.scatter(linear_shift_df.query('area=="MOs"')['n_units'],linear_shift_df.query('area=="MOs"')['null_accuracy_median'],s=5,alpha=0.5)
ax.set_xlabel('n_units')

ax.set_ylabel('null_accuracy_median')
ax.set_xlim([0,350])
ax.set_title('MOs recordings; r='+str(np.round(r,2))+' p='+str(np.round(p,8)))


In [ ]:
#plot null median vs. n_units using Templeton units only

#get median by bins
n_bins=8
n_units_bins=np.linspace(20,400,n_bins)
binned_n_units=np.full(n_bins-1,np.nan)
for ii in range(n_bins-1):
    binned_n_units[ii]=np.nanmedian(linear_shift_df.query('project.str.contains("Templeton") & area !="all" & n_units>=@n_units_bins[@ii] & n_units<@n_units_bins[@ii+1]')['null_accuracy_median'].values)

#get pearson correlation
r,p=st.pearsonr(linear_shift_df.query('project.str.contains("Templeton") & area !="all"')['n_units'],linear_shift_df.query('project.str.contains("Templeton") & area !="all"')['null_accuracy_median'])

fig,ax=plt.subplots(1,1)
ax.plot(n_units_bins[:-1],binned_n_units,color='k',linewidth=2.0)
ax.scatter(linear_shift_df.query('project.str.contains("Templeton") & area !="all"')['n_units'],linear_shift_df.query('project.str.contains("Templeton") & area !="all"')['null_accuracy_median'],s=5,alpha=0.5)
ax.set_xlabel('n_units')

ax.set_ylabel('null_accuracy_median')
ax.set_xlim([0,350])
ax.set_title('r='+str(np.round(r,2))+' p='+str(np.round(p,8)))


In [ ]:
min_n_recs=4
#plot distribution of number of units across areas
fig,ax=plt.subplots(1,1,figsize=(12,6))
area_list=[]
area_n_mean=[]
area_n_sem=[]
area_n_expts=[]
pos_counter=0

areas=linear_shift_df['area'].unique()

for ia,aa in enumerate(areas):
    if aa!="all":
        # ax.boxplot(linear_shift_df.query('area==@aa')['n_units'],positions=[ia])
        if len(linear_shift_df.query('area==@aa')['n_units'])>=min_n_recs:
            # ax.bar(pos_counter,np.mean(linear_shift_df.query('area==@aa')['n_units']),yerr=st.sem(linear_shift_df.query('area==@aa')['n_units']),label=aa)
            area_n_mean.append(np.mean(linear_shift_df.query('area==@aa')['n_units']))
            area_n_sem.append(st.sem(linear_shift_df.query('area==@aa')['n_units']))
            area_list.append(aa)
            area_n_expts.append(len(linear_shift_df.query('area==@aa')['n_units']))
            pos_counter+=1

area_n_df=pd.DataFrame({
    'area':area_list,
    'n_units':area_n_mean,
    'std':area_n_sem,
    'n_expts':area_n_expts
})

area_n_df.sort_values(by='n_units',ascending=False,inplace=True)
area_n_df.reset_index(inplace=True)

for ia,aa in area_n_df.iterrows():
    ax.bar(ia,aa['n_units'],yerr=aa['std'],label=aa['area'])

# ax.set_xticks(np.arange(len(area_list)))
# ax.set_xticklabels(area_n_df['area'],rotation=90)

area_labels=[]
for i in range(area_n_df.shape[0]):
    area_labels.append(area_n_df['area'].iloc[i]+' ('+str(area_n_df['n_expts'].iloc[i])+')')
ax.set_xticks(np.arange(len(area_n_df)))
ax.set_xticklabels(area_labels,rotation=90,ha='center')

ax.set_ylabel('n_units')
fig.suptitle('n units per probe per area for all experiments')
fig.tight_layout()


In [ ]:
min_n_recs=4
#plot distribution of number of units across areas
fig,ax=plt.subplots(1,1,figsize=(12,6))
area_list=[]
area_n_mean=[]
area_n_sem=[]
area_n_null_median=[]
area_n_diff_from_null=[]
area_n_true_acc=[]
area_n_expts=[]
pos_counter=0
areas=linear_shift_df['area'].unique()
for ia,aa in enumerate(areas):
    if aa!="all":
        # ax.boxplot(linear_shift_df.query('area==@aa')['n_units'],positions=[ia])
        if len(linear_shift_df.query('area==@aa')['n_units'])>=min_n_recs:
            # ax.bar(pos_counter,np.mean(linear_shift_df.query('area==@aa')['n_units']),yerr=st.sem(linear_shift_df.query('area==@aa')['n_units']),label=aa)
            area_n_mean.append(np.mean(linear_shift_df.query('area==@aa')['n_units']))
            area_n_sem.append(st.sem(linear_shift_df.query('area==@aa')['n_units']))
            area_n_null_median.append(np.median(linear_shift_df.query('area==@aa')['null_accuracy_median']))
            area_n_diff_from_null.append(np.mean(linear_shift_df.query('area==@aa')['true_accuracy'])-np.median(linear_shift_df.query('area==@aa')['null_accuracy_median']))
            area_n_true_acc.append(np.mean(linear_shift_df.query('area==@aa and project=="DynamicRouting"')['true_accuracy']))
            area_list.append(aa)
            area_n_expts.append(len(linear_shift_df.query('area==@aa')['n_units']))
            pos_counter+=1

area_n_df=pd.DataFrame({
    'area':area_list,
    'n_units':area_n_mean,
    'sem':area_n_sem,
    'null_accuracy_median':area_n_null_median,
    'diff_from_null':area_n_diff_from_null,
    'true_accuracy':area_n_true_acc,
    'n_expts':area_n_expts
})

area_n_df.sort_values(by='n_units',ascending=False,inplace=True)
area_n_df.reset_index(inplace=True)

ax2 = ax.twinx() 
ax2.set_ylim([0.5,0.85])
ax2.set_ylabel('decoder accuracy median')
#change axis color
ax2.spines['right'].set_color('red')
ax2.yaxis.label.set_color('red')
ax2.tick_params(axis='y', colors='red')

for ia,aa in area_n_df.iterrows():
    # ax.bar(ia,aa['n_units'],yerr=aa['sem'],label=aa['area'])
    ax.bar(ia,aa['n_units'],label=aa['area'],color='grey')

    ax2.plot(ia,aa['null_accuracy_median'],'ko',label='null')
    ax2.plot(ia,aa['true_accuracy'],'ro',label='true (DR only)')

    if ia==0:
        ax2.legend()
# ax.set_xticks(np.arange(len(area_list)))
# ax.set_xticklabels(area_n_df['area'],rotation=90)

area_labels=[]
for i in range(area_n_df.shape[0]):
    area_labels.append(area_n_df['area'].iloc[i]+' ('+str(area_n_df['n_expts'].iloc[i])+')')
ax.set_xticks(np.arange(len(area_n_df)))
ax.set_xticklabels(area_labels,rotation=90,ha='center')

ax.set_ylabel('n_units')
fig.suptitle('n units per probe per area for all experiments')
fig.tight_layout()


In [ ]:
#n units per probe versus difference from null

fig,ax=plt.subplots(1,1,figsize=(6,6))
ax.scatter(area_n_df['n_units'],area_n_df['diff_from_null'])
ax.set_xlabel('n_units')
ax.set_ylabel('true_accuracy - null_accuracy_median')
ax.set_title('n_units vs. true_accuracy - null_accuracy_median')

#n units per probe versus null

fig,ax=plt.subplots(1,1,figsize=(6,6))
ax.scatter(area_n_df['n_units'],area_n_df['null_accuracy_median'])
ax.set_xlabel('n_units')
ax.set_ylabel('null_accuracy_median')
ax.set_title('n_units vs. null_accuracy_median')


In [ ]:
area_n_df

In [ ]:
n_units_bins

In [ ]:
#compare DR and Templeton:
p_threshold=0.05

DR_linear_shift_df=linear_shift_df.query('project=="DynamicRouting" and cross_modal_dprime>=1.5')
#fraction significant
frac_sig_DR={
    'area':[],
    'frac_sig_DR':[],
    'n_expts_DR':[],
}
for area in DR_linear_shift_df['area'].unique():
    frac_sig_DR['area'].append(area)
    frac_sig_DR['frac_sig_DR'].append(np.mean(DR_linear_shift_df.query('area==@area')['p_value']<p_threshold))
    frac_sig_DR['n_expts_DR'].append(len(DR_linear_shift_df.query('area==@area')))
frac_sig_DR_df=pd.DataFrame(frac_sig_DR)
#diff from null
diff_from_null_DR={
    'area':[],
    'diff_from_null_mean_DR':[],
    'diff_from_null_median_DR':[],
    'true_accuracy_DR':[],
    'null_median_DR':[],
    'n_expts_DR':[],
}
for area in DR_linear_shift_df['area'].unique():
    diff_from_null_DR['area'].append(area)
    diff_from_null_DR['diff_from_null_mean_DR'].append((DR_linear_shift_df.query('area==@area')['true_accuracy']-
                                                 DR_linear_shift_df.query('area==@area')['null_accuracy_mean']).mean())
    diff_from_null_DR['diff_from_null_median_DR'].append((DR_linear_shift_df.query('area==@area')['true_accuracy']-
                                                    DR_linear_shift_df.query('area==@area')['null_accuracy_median']).median())
    diff_from_null_DR['true_accuracy_DR'].append(DR_linear_shift_df.query('area==@area')['true_accuracy'].median())
    diff_from_null_DR['null_median_DR'].append(DR_linear_shift_df.query('area==@area')['null_accuracy_median'].median())
    diff_from_null_DR['n_expts_DR'].append(len(DR_linear_shift_df.query('area==@area')))

diff_from_null_DR_df=pd.DataFrame(diff_from_null_DR)
diff_from_null_DR_df


Templeton_linear_shift_df=linear_shift_df.query('project.str.contains("Templeton")')
#fraction significant
frac_sig_Templ={
    'area':[],
    'frac_sig_Templ':[],
    'n_expts_Templ':[],
}
for area in Templeton_linear_shift_df['area'].unique():
    frac_sig_Templ['area'].append(area)
    frac_sig_Templ['frac_sig_Templ'].append(np.mean(Templeton_linear_shift_df.query('area==@area')['p_value']<p_threshold))
    frac_sig_Templ['n_expts_Templ'].append(len(Templeton_linear_shift_df.query('area==@area')))
frac_sig_Templ_df=pd.DataFrame(frac_sig_Templ)
#diff from null
diff_from_null_Templ={
    'area':[],
    'diff_from_null_mean_Templ':[],
    'diff_from_null_median_Templ':[],
    'true_accuracy_Templ':[],
    'null_median_Templ':[],
    'n_expts_Templ':[],
}
for area in Templeton_linear_shift_df['area'].unique():
    diff_from_null_Templ['area'].append(area)
    diff_from_null_Templ['diff_from_null_mean_Templ'].append((Templeton_linear_shift_df.query('area==@area')['true_accuracy']-
                                                 Templeton_linear_shift_df.query('area==@area')['null_accuracy_mean']).mean())
    diff_from_null_Templ['diff_from_null_median_Templ'].append((Templeton_linear_shift_df.query('area==@area')['true_accuracy']-
                                                    Templeton_linear_shift_df.query('area==@area')['null_accuracy_median']).median())
    diff_from_null_Templ['true_accuracy_Templ'].append(Templeton_linear_shift_df.query('area==@area')['true_accuracy'].median())
    diff_from_null_Templ['null_median_Templ'].append(Templeton_linear_shift_df.query('area==@area')['null_accuracy_median'].median())
    diff_from_null_Templ['n_expts_Templ'].append(len(Templeton_linear_shift_df.query('area==@area')))
diff_from_null_Templ_df=pd.DataFrame(diff_from_null_Templ)


all_frac_sig_df=pd.merge(frac_sig_DR_df,frac_sig_Templ_df,on='area',how='outer')
all_diff_from_null_df=pd.merge(diff_from_null_DR_df,diff_from_null_Templ_df,on='area',how='outer')

In [ ]:
# np.mean(Templeton_linear_shift_df['p_value']<p_threshold)
DR_linear_shift_df

In [ ]:
fig,ax=plt.subplots(2,1,figsize=(5,5))
ax[0].hist(DR_linear_shift_df['true_accuracy']-DR_linear_shift_df['null_accuracy_median'],bins=20)
ax[0].set_xlabel('diff from null median')
ax[1].hist(DR_linear_shift_df['p_value'],bins=20)
ax[1].set_xlabel('linear shift derived p-value')

fig.suptitle('context decoding accuracy with 20 facemap SVDs')
fig.tight_layout()

In [ ]:
#barplot sorted by fraction significant
min_n_expts=4

plot_DR_frac_sig_df=frac_sig_DR_df.query('n_expts_DR>=@min_n_expts').sort_values('frac_sig_DR',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(8,4))
plot_DR_frac_sig_df.plot.bar(x='area',y=['frac_sig_DR'],ax=ax)
ax.set_ylabel('fraction significant decoding')

area_labels=[]
for i in range(plot_DR_frac_sig_df.shape[0]):
    area_labels.append(plot_DR_frac_sig_df['area'].iloc[i]+' ('+str(plot_DR_frac_sig_df['n_expts_DR'].iloc[i])+')')
ax.set_xticklabels(area_labels,rotation=90,ha='center')
ax.set_ylim([0,1])
fig.tight_layout()

In [ ]:
#barplot sorted by DR diff from median
min_n_expts=4

plot_DR_diff_from_null_df=diff_from_null_DR_df.query('n_expts_DR>=@min_n_expts').sort_values('diff_from_null_median_DR',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(8,4))
plot_DR_diff_from_null_df.plot.bar(x='area',y=['diff_from_null_median_DR'],ax=ax)

ax.set_ylabel('median difference from null')
area_labels=[]
for i in range(plot_DR_diff_from_null_df.shape[0]):
    area_labels.append(plot_DR_diff_from_null_df['area'].iloc[i]+' ('+str(plot_DR_diff_from_null_df['n_expts_DR'].iloc[i])+')')
ax.set_xticklabels(area_labels,rotation=90,ha='center')

ax.set_ylim([0,0.20])
fig.tight_layout()



In [ ]:
#plot null median across areas
min_n_expts=4

plot_DR_diff_from_null_df=diff_from_null_DR_df.query('n_expts_DR>=@min_n_expts').sort_values('diff_from_null_median_DR',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(8,4))
plot_DR_diff_from_null_df.plot.bar(x='area',y=['null_median_DR'],ax=ax)

ax.set_ylabel('null median')
area_labels=[]
for i in range(plot_DR_diff_from_null_df.shape[0]):
    area_labels.append(plot_DR_diff_from_null_df['area'].iloc[i]+' ('+str(plot_DR_diff_from_null_df['n_expts_DR'].iloc[i])+')')
ax.set_xticklabels(area_labels,rotation=90,ha='center')

ax.set_ylim([0.5,1.0])
fig.tight_layout()


In [ ]:
#plot true accuracy and null median on same barplot

min_n_expts=4

plot_DR_diff_from_null_df=diff_from_null_DR_df.query('n_expts_DR>=@min_n_expts').sort_values('diff_from_null_median_DR',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(8,4))
plot_DR_diff_from_null_df.plot.bar(x='area',y=['true_accuracy_DR','null_median_DR'],ax=ax)

ax.set_ylabel('accuracy')
area_labels=[]
for i in range(plot_DR_diff_from_null_df.shape[0]):
    area_labels.append(plot_DR_diff_from_null_df['area'].iloc[i]+' ('+str(plot_DR_diff_from_null_df['n_expts_DR'].iloc[i])+')')

ax.set_xticklabels(area_labels,rotation=90,ha='center')

ax.set_ylim([0.5,1.0])
fig.tight_layout()

In [ ]:
#barplot sorted by fraction significant
min_n_expts=2

plot_all_frac_sig_df=all_frac_sig_df.query('n_expts_DR>=@min_n_expts and n_expts_Templ>=@min_n_expts').sort_values('frac_sig_DR',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(8,4))
plot_all_frac_sig_df.plot.bar(x='area',y=['frac_sig_DR','frac_sig_Templ'],ax=ax)
ax.set_ylabel('fraction significant decoding')
# ax.set_xlabel('area')
ax.legend(['DR','Templ'])
ax.set_ylim([0,1])
fig.tight_layout()

In [ ]:
#barplot sorted by DR diff from median
min_n_expts=2

plot_all_diff_from_null_df=all_diff_from_null_df.query('n_expts_DR>=@min_n_expts and n_expts_Templ>=@min_n_expts').sort_values('diff_from_null_median_DR',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(8,4))
plot_all_diff_from_null_df.plot.bar(x='area',y=['diff_from_null_median_DR','diff_from_null_median_Templ'],ax=ax)

ax.set_ylabel('median difference from null')
# ax.set_xlabel('area')
ax.legend(['DR','Templ'])

ax.set_ylim([-0.03,0.18])
fig.tight_layout()



In [ ]:
x_vect=np.arange(0,1.01,0.05)

fig,ax=plt.subplots(1,1)
ax.hist(1-DR_linear_shift_df['p_value'],bins=x_vect,alpha=0.5)
ax.hist(1-Templeton_linear_shift_df['p_value'],bins=x_vect,alpha=0.5)

ax.set_xlabel('zero shift percentile')
ax.set_ylabel('count (session-areas)')

ax.legend(['DR','Templeton'])



# fig,ax=plt.subplots(1,1)
# ax.hist(1-DR_linear_shift_df['p_value'],bins=x_vect,alpha=0.5,density=True)
# ax.hist(1-Templeton_linear_shift_df['p_value'],bins=x_vect,alpha=0.5,density=True)

# ax.set_xlabel('zero shift percentile')
# ax.set_ylabel('fraction of session-areas')

# ax.legend(['DR','Templeton'])

In [ ]:
x_vect=np.arange(-0.1,0.3,0.02)

fig,ax=plt.subplots(1,1)
ax.hist(DR_linear_shift_df['true_accuracy']-DR_linear_shift_df['null_accuracy_median'],bins=x_vect,alpha=0.5)
ax.hist(Templeton_linear_shift_df['true_accuracy']-Templeton_linear_shift_df['null_accuracy_median'],bins=x_vect,alpha=0.5)

ax.set_xlabel('difference from null distribution median')
ax.set_ylabel('count (session-areas)')

ax.legend(['DR','Templeton'])

In [ ]:
all_unit_linear_shift=pd.read_csv(r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\new_annotations\linear_shift_20_units\combined\all_unit_linear_shift_use_more_trials.csv")
all_facemap_linear_shift=pd.read_csv(r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\new_annotations\linear_shift_facemap_face_use_more_trials_500_svds\combined\all_unit_linear_shift_use_more_trials.csv")

all_unit_linear_shift['diff_from_null_median']=all_unit_linear_shift['true_accuracy']-all_unit_linear_shift['null_accuracy_median']
for rr,row in all_unit_linear_shift.iterrows():
    if row['session_id'][-2:]=='_0':
        all_unit_linear_shift.loc[rr,'session_id']=row['session_id'][:-2]
all_facemap_linear_shift['diff_from_null_median']=all_facemap_linear_shift['true_accuracy']-all_facemap_linear_shift['null_accuracy_median']
for rr,row in all_facemap_linear_shift.iterrows():
    if row['session_id'][-2:]=='_0':
        all_facemap_linear_shift.loc[rr,'session_id']=row['session_id'][:-2]

all_linear_shift_df=pd.merge(all_unit_linear_shift,all_facemap_linear_shift,on=['session_id','project'],suffixes=('_units','_face'),how='outer')
all_linear_shift_df

In [ ]:
# all_facemap_linear_shift

In [ ]:
#compare facemap to unit decoders

#load unit decoding_results
DR_loadpath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\linear_shift\combined\DR_linear_shift_results.csv"
Templeton_loadpath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\linear_shift\combined\Templ_linear_shift_results.csv"
DR_unit_linear_shift_df=pd.read_csv(DR_loadpath)
DR_unit_linear_shift_df['project']='DynamicRouting'
DR_unit_linear_shift_df['diff_from_null_median']=DR_unit_linear_shift_df['true_accuracy']-DR_unit_linear_shift_df['null_accuracy_median']
Templeton_unit_linear_shift_df=pd.read_csv(Templeton_loadpath)
Templeton_unit_linear_shift_df['project']='TempletonPilotSession'
Templeton_unit_linear_shift_df['diff_from_null_median']=Templeton_unit_linear_shift_df['true_accuracy']-Templeton_unit_linear_shift_df['null_accuracy_median']

#facemap decoding results
facemap_face_loadpath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\linear_shift\combined\all_facemap_face_linear_shift_results.csv"
facemap_behavior_loadpath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\linear_shift\combined\all_facemap_behavior_linear_shift_results.csv"
facemap_face_linear_shift_df=pd.read_csv(facemap_face_loadpath)
facemap_face_linear_shift_df['diff_from_null_median']=facemap_face_linear_shift_df['true_accuracy']-facemap_face_linear_shift_df['null_accuracy_median']
facemap_behavior_linear_shift_df=pd.read_csv(facemap_behavior_loadpath)
facemap_behavior_linear_shift_df['diff_from_null_median']=facemap_behavior_linear_shift_df['true_accuracy']-facemap_behavior_linear_shift_df['null_accuracy_median']



In [ ]:
# DR_unit_linear_shift_df
# type(all_facemap_linear_shift_df['project'].unique()[1])
# type(all_unit_linear_shift_df['project'].unique()[0])
DR_unit_linear_shift_df['session_id']

In [ ]:
Templeton_unit_linear_shift_df['session_id']

In [ ]:
#merge all dfs
all_unit_linear_shift_df=pd.concat([DR_unit_linear_shift_df,Templeton_unit_linear_shift_df],axis=0).reset_index()
for rr,row in all_unit_linear_shift_df.iterrows():
    if row['session_id'][-2:]=='_0':
        all_unit_linear_shift_df.loc[rr,'session_id']=row['session_id'][:-2]

all_facemap_linear_shift_df=pd.merge(facemap_face_linear_shift_df,facemap_behavior_linear_shift_df,on=['session_id','area','project'],suffixes=('_face','_behavior'))

all_linear_shift_df=pd.merge(all_unit_linear_shift_df,all_facemap_linear_shift_df,on=['session_id','project'],suffixes=('_units','_facemap'),how='outer')
all_linear_shift_df

In [ ]:
# all_facemap_linear_shift_df['project']
# 

In [ ]:
all_linear_shift_df['session_id']

In [ ]:
# facemap_face_linear_shift_df['session_id'].values

# all_DR_linear_shift_df

In [ ]:
all_DR_linear_shift_df=all_linear_shift_df.query('project=="DynamicRouting" and area_units.isna()==False and cross_modal_dprime_units>=1.0')

diff_from_null_DR={
    'area':[],
    'sessions':[],
    'DR_diff_from_null_median':[],
    'DR_diff_from_null_sem':[],
    'facemap_face_diff_from_null_median':[],
    'facemap_face_diff_from_null_sem':[],
    # 'facemap_behavior_diff_from_null_median':[],
    'n_expts_DR':[],
    'n_expts_facemap_face':[],
    # 'n_expts_facemap_behavior':[],
}
for area in all_DR_linear_shift_df['area_units'].unique():
    incl_sessions=all_DR_linear_shift_df.query('area_units==@area')['session_id'].values

    diff_from_null_DR['area'].append(area)
    diff_from_null_DR['sessions'].append(incl_sessions)
    diff_from_null_DR['DR_diff_from_null_median'].append((all_DR_linear_shift_df.query('area_units==@area')['diff_from_null_median_units'].mean()))
    diff_from_null_DR['DR_diff_from_null_sem'].append((all_DR_linear_shift_df.query('area_units==@area')['diff_from_null_median_units'].sem()))
    diff_from_null_DR['n_expts_DR'].append(len(all_DR_linear_shift_df.query('area_units==@area')))

    face=[]
    # behavior=[]
    for ss in incl_sessions:
        face.append(all_DR_linear_shift_df.query('session_id==@ss')['diff_from_null_median_face'].mean())
        # behavior.append(all_DR_linear_shift_df.query('session_id==@ss')['diff_from_null_median_behavior'].mean())
    
    diff_from_null_DR['facemap_face_diff_from_null_median'].append(np.nanmean(face))
    diff_from_null_DR['facemap_face_diff_from_null_sem'].append(st.sem(face,nan_policy='omit'))

    # diff_from_null_DR['facemap_behavior_diff_from_null_median'].append(np.nanmean(behavior))

    diff_from_null_DR['n_expts_facemap_face'].append(np.sum(np.isnan(face)==False))
    # diff_from_null_DR['n_expts_facemap_behavior'].append(np.sum(np.isnan(behavior)==False))


diff_from_null_DR_df=pd.DataFrame(diff_from_null_DR)
diff_from_null_DR_df

In [ ]:
# all_DR_linear_shift_df.query('session_id==@ss')
all_DR_linear_shift_df['area_units']

In [ ]:
min_n_expts=4
sorted_df=diff_from_null_DR_df.query('n_expts_DR>=@min_n_expts and n_expts_facemap_face>=@min_n_expts and n_expts_facemap_behavior>=@min_n_expts').sort_values('DR_diff_from_null_median',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(12,6))
sorted_df.plot.bar(x='area',y=['DR_diff_from_null_median'],ax=ax,alpha=0.5)

ax.plot(np.arange(0,len(sorted_df)),sorted_df[['facemap_face_diff_from_null_median','facemap_behavior_diff_from_null_median']].mean(axis=1),color='r',linestyle='',marker='_',label='facemap_mean')

area_labels=[]
for i in range(sorted_df.shape[0]):
    area_labels.append(sorted_df['area'].iloc[i]+' ('+str(sorted_df['n_expts_DR'].iloc[i])+')')
ax.set_xticklabels(area_labels)
ax.set_ylabel('median difference from null')
ax.set_ylim([-0.01,0.12])
ax.legend()

In [ ]:
min_n_expts=3
sorted_df=diff_from_null_DR_df.query('n_expts_DR>=@min_n_expts and n_expts_facemap_face>=@min_n_expts').sort_values('DR_diff_from_null_median',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(12,6))
sorted_df.plot.bar(x='area',y=['DR_diff_from_null_median'],ax=ax,alpha=0.5)

ax.plot(np.arange(0,len(sorted_df)),sorted_df[['facemap_face_diff_from_null_median']].mean(axis=1),color='r',linestyle='',marker='_',label='facemap_mean')

area_labels=[]
for i in range(sorted_df.shape[0]):
    area_labels.append(sorted_df['area'].iloc[i]+' ('+str(sorted_df['n_expts_DR'].iloc[i])+')')
ax.set_xticklabels(area_labels)
ax.set_ylabel('median difference from null')
ax.set_ylim([-0.01,0.18])
ax.legend()

In [ ]:
min_n_expts=3
area_list='ORB|ACA|MOs|VISp|FRP|LGd|MGd|CA1|SC|LP|CP|MRN'
sorted_df=diff_from_null_DR_df.query('n_expts_DR>=@min_n_expts and n_expts_facemap_face>=@min_n_expts and area.str.contains(@area_list)').sort_values('DR_diff_from_null_median',ascending=False)

facemap_df=facemap_df=pd.DataFrame({
    'area':['video'],
    'DR_diff_from_null_median':diff_from_null_DR_df.query('area=="all"')['facemap_face_diff_from_null_median'].values,
    'DR_diff_from_null_sem':diff_from_null_DR_df.query('area=="all"')['facemap_face_diff_from_null_sem'].values,
    'n_expts_DR':diff_from_null_DR_df.query('area=="all"')['n_expts_facemap_face'].values
})

sorted_df=pd.concat([sorted_df,facemap_df],axis=0)

fig,ax=plt.subplots(1,1,figsize=(10,5))
sorted_df.plot.bar(x='area',y=['DR_diff_from_null_median'],ax=ax,alpha=0.5,legend=False)
ax.errorbar(np.arange(0,len(sorted_df)),sorted_df['DR_diff_from_null_median'],yerr=sorted_df['DR_diff_from_null_sem'],linestyle='',color='k')

# ax.plot(np.arange(0,len(sorted_df)),sorted_df[['facemap_face_diff_from_null_median']].mean(axis=1),color='r',linestyle='',marker='_',label='facemap_mean')

area_labels=[]
for i in range(sorted_df.shape[0]):
    area_labels.append(sorted_df['area'].iloc[i]+' ('+str(sorted_df['n_expts_DR'].iloc[i])+')')
ax.set_xticklabels(area_labels)
ax.set_ylabel('context decoding accuracy above null')
ax.set_xlabel('')
ax.set_ylim([0,0.19])
# ax.legend()

In [ ]:
# diff_from_null_DR_df.query('area=="all"')
# sorted_df
diff_from_null_DR_df.query('area=="all"')['facemap_face_diff_from_null_sem'].values

In [ ]:
facemap_df=pd.DataFrame({
    'area':['video'],
    'DR_diff_from_null_median':[all_DR_linear_shift_df['diff_from_null_median_face'].mean()],
    'DR_diff_from_null_sem':[all_DR_linear_shift_df['diff_from_null_median_face'].sem()],

})
pd.concat([sorted_df,facemap_df],axis=0)

In [ ]:
all_templeton_linear_shift_df=all_linear_shift_df.query('project.str.contains("Templeton") and area_units.isna()==False')

diff_from_null_Templ={
    'area':[],
    'sessions':[],
    'Templ_diff_from_null_median':[],
    'facemap_face_diff_from_null_median':[],
    'facemap_behavior_diff_from_null_median':[],
    'n_expts_Templ':[],
}

for area in all_templeton_linear_shift_df['area_units'].unique():
    incl_sessions=all_templeton_linear_shift_df.query('area_units==@area')['session_id'].values
    diff_from_null_Templ['area'].append(area)
    diff_from_null_Templ['sessions'].append(incl_sessions)
    diff_from_null_Templ['Templ_diff_from_null_median'].append((all_templeton_linear_shift_df.query('area_units==@area')['diff_from_null_median'].mean()))
    diff_from_null_Templ['n_expts_Templ'].append(len(all_templeton_linear_shift_df.query('area_units==@area')))

    diff_from_null_Templ['facemap_face_diff_from_null_median'].append(all_templeton_linear_shift_df.query('area_units==@area')['diff_from_null_median_face'].mean())
    diff_from_null_Templ['facemap_behavior_diff_from_null_median'].append(all_templeton_linear_shift_df.query('area_units==@area')['diff_from_null_median_behavior'].mean())

diff_from_null_Templ_df=pd.DataFrame(diff_from_null_Templ)
diff_from_null_Templ_df



In [ ]:
min_n_expts=2

sorted_df=diff_from_null_Templ_df.query('n_expts_Templ>=@min_n_expts').sort_values('Templ_diff_from_null_median',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(12,6))
sorted_df.plot.bar(x='area',y=['Templ_diff_from_null_median'],ax=ax,alpha=0.5)

ax.plot(np.arange(0,len(sorted_df)),sorted_df[['facemap_face_diff_from_null_median','facemap_behavior_diff_from_null_median']].mean(axis=1),color='r',linestyle='',marker='_',label='facemap_mean')

area_labels=[]
for i in range(sorted_df.shape[0]):
    area_labels.append(sorted_df['area'].iloc[i]+' ('+str(sorted_df['n_expts_Templ'].iloc[i])+')')
ax.set_xticklabels(area_labels)
ax.set_ylabel('median difference from null')
ax.set_ylim([-0.06,0.07])
ax.legend()

In [ ]:
#session-wise correlation with behavior performance

all_linear_shift=pd.read_csv(r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\new_annotations\linear_shift_20_units\combined\all_unit_linear_shift_use_more_trials.csv")

In [ ]:
all_linear_shift.columns

In [ ]:
DR_whole_sessions=all_linear_shift.query('project=="DynamicRouting" and area=="all"')

r,p=st.pearsonr(DR_whole_sessions['cross_modal_dprime'],DR_whole_sessions['true_accuracy']-DR_whole_sessions['null_accuracy_median'])

#fit line
slope,intercept,r,p,stderr=st.linregress(DR_whole_sessions['cross_modal_dprime'],DR_whole_sessions['true_accuracy']-DR_whole_sessions['null_accuracy_median'])

fig,ax=plt.subplots(1,1,figsize=(5,5))

ax.plot(DR_whole_sessions['cross_modal_dprime'],DR_whole_sessions['true_accuracy']-DR_whole_sessions['null_accuracy_median'],'k.')
ax.plot(DR_whole_sessions['cross_modal_dprime'],slope*DR_whole_sessions['cross_modal_dprime']+intercept,'grey',alpha=0.5)
ax.set_xlabel('session dprime')
ax.set_ylabel('decoding accuracy above null')
ax.set_title('all units; r='+str(np.round(r,2))+' p='+str(np.round(p,10)))
#remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

In [ ]:
area_comps=['ORBl','DG']
colors=['tab:red','tab:green']

fig,ax=plt.subplots(1,1,figsize=(5,5))

for ia,aa in enumerate(area_comps):

    DR_whole_sessions=all_linear_shift.query('project=="DynamicRouting" and area==@aa')

    r,p=st.pearsonr(DR_whole_sessions['cross_modal_dprime'],DR_whole_sessions['true_accuracy']-DR_whole_sessions['null_accuracy_median'])

    #fit line
    slope,intercept,r_,p_,stderr=st.linregress(DR_whole_sessions['cross_modal_dprime'],DR_whole_sessions['true_accuracy']-DR_whole_sessions['null_accuracy_median'])

    ax.plot(DR_whole_sessions['cross_modal_dprime'],DR_whole_sessions['true_accuracy']-DR_whole_sessions['null_accuracy_median'],color=colors[ia],linestyle='',marker='.',alpha=0.5)
    ax.plot(DR_whole_sessions['cross_modal_dprime'],slope*DR_whole_sessions['cross_modal_dprime']+intercept,color=colors[ia],alpha=0.5)

    ax.set_xlabel('session dprime')
    ax.set_ylabel('decoding accuracy above null')
    ax.text(-0.3,0.16-ia*0.013,aa+' r='+str(np.round(r,2))+' p='+str(np.round(p,4)),color=colors[ia],fontsize=12)
#remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

In [ ]:
decoder_conf_by_block=pd.read_csv(r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\new_annotations\linear_shift_20_units\combined\decoder_confidence_dprime_by_block.csv")
decoder_conf_by_block

In [ ]:
dr_conf_by_block=decoder_conf_by_block.query('project=="DynamicRouting"')

area_conf_dprime_corr={
    'area':[],
    'r':[],
    'p':[],
    'n_blocks':[],
}

for aa in dr_conf_by_block['area'].unique():
    dprime=dr_conf_by_block.query('area==@aa')['cross_modal_dprime'].values
    conf=dr_conf_by_block.query('area==@aa')['confidence'].values
    #remove nans
    dprime_nans=np.isnan(dprime)|np.isinf(dprime)
    conf_nans=np.isnan(conf)|np.isinf(conf)
    dprime=dprime[(~dprime_nans)&(~conf_nans)]
    conf=conf[(~dprime_nans)&(~conf_nans)]

    r,p=st.pearsonr(dprime,conf)
    area_conf_dprime_corr['area'].append(aa)
    area_conf_dprime_corr['r'].append(r)
    area_conf_dprime_corr['p'].append(p)
    area_conf_dprime_corr['n_blocks'].append(len(dr_conf_by_block.query('area==@aa')))

area_conf_dprime_corr_df=pd.DataFrame(area_conf_dprime_corr)
area_conf_dprime_corr_df

In [ ]:
sorted=area_conf_dprime_corr_df.copy()
# sorted['r']=np.abs(sorted['r'])

sorted=sorted.sort_values('r',ascending=True)
sorted[:50]

In [ ]:
facemap_and_spikes_shift=pd.read_csv(r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\new_annotations\linear_shift_spikes_and_facemap_face_more_trials_20_svds\combined\facemap_and_spikes_shift_results.csv")
facemap_and_spikes_shift['diff_from_null_spikes_median']=facemap_and_spikes_shift['true_accuracy_spikes']-facemap_and_spikes_shift['null_accuracy_spikes_median']
facemap_and_spikes_shift['diff_from_null_facemap_median']=facemap_and_spikes_shift['true_accuracy_facemap']-facemap_and_spikes_shift['null_accuracy_facemap_median']
facemap_and_spikes_shift

In [ ]:
all_DR_spike_facemap_shift_df=facemap_and_spikes_shift.query('project=="DynamicRouting" and cross_modal_dprime>=1.0')

diff_from_null_spike_facemap={
    'area':[],
    'sessions':[],
    'spikes_diff_from_null_median':[],
    'spikes_diff_from_null_sem':[],
    'facemap_diff_from_null_median':[],
    'facemap_diff_from_null_sem':[],

    'n_expts':[],

}
for area in all_DR_spike_facemap_shift_df['area'].unique():
    incl_sessions=all_DR_spike_facemap_shift_df.query('area==@area')['session'].values

    diff_from_null_spike_facemap['area'].append(area)
    area_sessions=all_DR_spike_facemap_shift_df.query('area==@area')

    diff_from_null_spike_facemap['sessions'].append(incl_sessions)
    diff_from_null_spike_facemap['spikes_diff_from_null_median'].append((area_sessions['diff_from_null_spikes_median'].mean()))
    diff_from_null_spike_facemap['spikes_diff_from_null_sem'].append((area_sessions['diff_from_null_spikes_median'].sem()))
    diff_from_null_spike_facemap['facemap_diff_from_null_median'].append((area_sessions['diff_from_null_facemap_median'].mean()))
    diff_from_null_spike_facemap['facemap_diff_from_null_sem'].append((area_sessions['diff_from_null_facemap_median'].sem()))

    diff_from_null_spike_facemap['n_expts'].append(len(area_sessions))


diff_from_null_spike_facemap=pd.DataFrame(diff_from_null_spike_facemap)
diff_from_null_spike_facemap

In [ ]:
sel_area_list='ORB|ACA|MOs|VISp|FRP|LGd|MGd|CA1|SC|LP|CP|MRN'

sorted_df=diff_from_null_spike_facemap.query('n_expts>=3 and area.str.contains(@sel_area_list)').sort_values('spikes_diff_from_null_median',ascending=False)

fig,ax=plt.subplots(1,1,figsize=(10,5))
sorted_df.plot.bar(x='area',y=['spikes_diff_from_null_median'],ax=ax,alpha=0.5,legend=False)
ax.errorbar(np.arange(0,len(sorted_df)),sorted_df['spikes_diff_from_null_median'],yerr=sorted_df['spikes_diff_from_null_sem'],linestyle='',color='k')

ax.set_ylabel('context decoding accuracy above video null')
ax.set_xlabel('')

In [2]:
session=DynamicRoutingSession('712141_2024-06-06')

In [3]:
session.trials[:]

,start_time,stop_time,quiescent_start_time,quiescent_stop_time,stim_start_time,stim_stop_time,response_window_start_time,response_window_stop_time,task_control_response_time,response_time,...,is_aud_target,is_vis_target,is_nontarget,is_aud_nontarget,is_vis_nontarget,is_vis_context,is_aud_context,is_context_switch,is_repeat,is_opto
id,,,,,,,,,,,,,,,,,,,,,
0,928.73316,934.25448,928.73316,930.23438,930.242420,930.742420,930.31783,931.23527,930.71817,930.71457,...,True,False,False,False,False,False,True,False,False,False
1,934.93837,940.44297,934.93837,936.43964,936.447730,936.947730,936.52331,937.44048,936.87330,936.86431,...,True,False,False,False,False,False,True,False,False,False
2,940.54307,946.04769,940.54307,942.04431,942.052230,942.552230,942.12772,943.04515,942.46135,942.44600,...,True,False,False,False,False,False,True,False,False,False
3,948.31627,953.82086,948.31627,949.81749,949.825320,950.325320,949.90089,950.81834,950.06771,950.06171,...,True,False,False,False,False,False,True,False,False,False
4,954.32143,959.82588,954.32143,955.82252,955.830300,956.330300,955.90614,956.82338,956.10610,956.10549,...,True,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,4529.50162,4535.00634,4529.50162,4531.00289,4531.010820,4531.510820,4531.08629,4532.00373,NaN,NaN,...,True,False,False,False,False,True,False,False,False,False
530,4535.60677,4541.12806,4535.60677,4537.10805,4537.138611,4537.639021,4537.19147,4538.10884,4537.89203,4537.88410,...,False,True,False,False,False,True,False,False,False,False
531,4541.17815,4546.68273,4541.17815,4542.67937,4542.687410,4543.187410,4542.76276,4543.68024,NaN,NaN,...,True,False,False,False,False,True,False,False,False,False


In [4]:
session.units[:]

fetching units: 100%|█████████████████████████| 6/6 [01:42<00:00, 17.01s/device]
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  return cls(**config)
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  return cls(**config)
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  return cls(**config)
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  

,amplitude_cutoff,amplitude_cv_median,amplitude_cv_range,amplitude_median,drift_ptp,drift_std,drift_mad,firing_range,firing_rate,isi_violations_ratio,...,electrode_group_name,peak_channel,cluster_id,default_qc,amplitude,unit_id,peak_electrode,spike_times,obs_intervals,electrode_group
id,,,,,,,,,,,,,,,,,,,,,
0,0.000308,0.260543,0.094237,74.880000,NaN,NaN,NaN,1.60,0.569180,0.000000,...,probeA,3,0,True,115.647472,712141_2024-06-06_A-0,3,"[21.654997379977058, 23.58042585684688, 27.363...","[(19.838868627423082, 5448.683875577795)]",probeA pynwb.ecephys.ElectrodeGroup at 0x27196...
1,0.000045,0.235215,0.108702,88.920000,11.678773,1.439521,2.390120,6.96,3.824008,0.100773,...,probeA,0,1,True,130.047836,712141_2024-06-06_A-1,0,"[22.056263034540688, 23.15472693057739, 23.262...","[(19.838868627423082, 5448.683875577795)]",probeA pynwb.ecephys.ElectrodeGroup at 0x27196...
2,0.000085,0.215543,0.173246,81.899994,11.022746,1.966640,2.367459,19.56,6.018208,0.086458,...,probeA,1,2,True,124.492634,712141_2024-06-06_A-2,1,"[19.956768330047403, 20.269767540576765, 20.36...","[(19.838868627423082, 5448.683875577795)]",probeA pynwb.ecephys.ElectrodeGroup at 0x27196...
3,0.000086,NaN,NaN,109.979996,NaN,NaN,NaN,12.00,2.746802,0.040690,...,probeA,2,3,True,159.475723,712141_2024-06-06_A-3,2,"[19.851735261636538, 20.049701428977738, 20.31...","[(19.838868627423082, 5448.683875577795)]",probeA pynwb.ecephys.ElectrodeGroup at 0x27196...
4,0.000089,0.171025,0.086723,93.600000,24.911341,3.288690,3.368261,7.80,3.704646,0.098424,...,probeA,4,4,True,138.701199,712141_2024-06-06_A-4,4,"[21.33579818508577, 22.824861095923655, 30.767...","[(19.838868627423082, 5448.683875577795)]",probeA pynwb.ecephys.ElectrodeGroup at 0x27196...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,0.000077,NaN,NaN,79.560000,NaN,NaN,NaN,3.60,1.149047,0.186018,...,probeF,311,873,True,119.901562,712141_2024-06-06_F-873,2231,"[39.36005641498136, 42.11241535616081, 90.5550...","[(19.850877841188016, 5448.684539863181)]",probeF pynwb.ecephys.ElectrodeGroup at 0x27196...
894,0.000163,0.231518,0.174987,79.560000,NaN,NaN,NaN,2.00,0.724647,0.000000,...,probeF,309,875,True,108.318541,712141_2024-06-06_F-875,2229,"[29.432617613249136, 47.853932573857065, 72.56...","[(19.850877841188016, 5448.684539863181)]",probeF pynwb.ecephys.ElectrodeGroup at 0x27196...
895,0.000787,0.254005,0.134215,44.460000,NaN,NaN,NaN,2.80,0.880297,0.000000,...,probeF,316,876,True,62.843060,712141_2024-06-06_F-876,2236,"[26.19249337449209, 26.365059556789582, 31.901...","[(19.850877841188016, 5448.684539863181)]",probeF pynwb.ecephys.ElectrodeGroup at 0x27196...


In [ ]:
all_DR_linear_shift_df['area_units']

In [ ]:
facemap_face_linear_shift_df.query('session_id==@ss')

In [ ]:
session_id=list(decoder_results.keys())[19]

decoder_results[session_id]['results'].keys()

In [ ]:
performance=pd.read_parquet(
                npc_lims.get_cache_path('performance',session_id,version='any')
            )
performance

In [ ]:
units.columns

In [ ]:
aa='MOs'
unit_ids=units[:].query('structure==@aa')['unit_id'].values
structure_probe.loc[structure_probe['unit_id'].isin(unit_ids),'structure_probe']=aa+'_'+'blah'

In [ ]:
structure_probe

In [ ]:
units=pd.read_parquet(
    npc_lims.get_cache_path('units','626791_2022-08-15',version='any')
)
session=DynamicRoutingSession('626791_2022-08-15')
structure_probe=spike_utils.get_structure_probe(session)
# units['structure']=structure_probe['structure_probe']

In [ ]:
units['structure_probe']=np.full(units.shape[0],'',dtype='object')
for uu, unit in units.iterrows():
    units.loc[units['unit_id']==unit['unit_id'],'structure_probe']=structure_probe.loc[structure_probe['unit_id']==unit['unit_id'],'structure_probe']
units[['structure','structure_probe','electrode_group_name']][600:650]

In [ ]:
#plot block cross-modal dprime versus mean decoder confidence?

#for each area

#correlation between dprime and decoder confidence for each area?
#areas with highest correlation probably are most important to task

#sub areas?? MOs, CP, etc.

In [ ]:
#concat all switches for each area
### how to normalize?? divide by stdev of confidence?

In [ ]:
#plot decoder confidence versus trials/time since last rewarded target?

#add column to trials table for time/trials since last rewarded target

#plot decoder confidence versus time since last rewarded target

In [ ]:
list(decoder_results.keys())[50]

In [ ]:
list(decoder_results.keys())[50]
decoder_results[list(decoder_results.keys())[50]]['results'].keys()

In [ ]:
#trial-trial decoding: decision function
session_id=list(decoder_results.keys())[50]
aa='ORBl'

shifts=decoder_results[session_id]['shifts']
areas=decoder_results[session_id]['areas']

half_neg_shift=np.round(shifts.min()/2)
half_pos_shift=np.round(shifts.max()/2)
# half_shifts=np.arange(-half_neg_shift,half_pos_shift+1)
half_neg_shift_ind=np.where(shifts==half_neg_shift)[0][0]
half_pos_shift_ind=np.where(shifts==half_pos_shift)[0][0]
half_shift_inds=np.arange(half_neg_shift_ind,half_pos_shift_ind+1)
if use_half_shifts:
    half_shift_inds=np.arange(len(shifts))

decision_function_shifts=[]

for sh in half_shift_inds:
    decision_function_shifts.append(decoder_results[session_id]['results'][aa]['shift'][sh]['decision_function'])

true_label=decoder_results[session_id]['results'][aa]['shift'][np.where(shifts==0)[0][0]]['true_label']

decision_function_shifts=np.vstack(decision_function_shifts)
decision_function_shifts

In [ ]:
fig,ax=plt.subplots(1,1)
ax.plot(decoder_results[session_id]['middle_4_blocks'],decision_function_shifts.T,color='grey',alpha=0.1)
ax.plot(decoder_results[session_id]['middle_4_blocks'],np.median(decision_function_shifts,axis=0),color='k',linewidth=2,label='median null decoding')
ax.plot(decoder_results[session_id]['middle_4_blocks'],true_label*15-7.5,'b--',linewidth=2,alpha=0.5,label='true block label')
ax.set_ylabel('decision function')
ax.set_xlabel('trial')
ax.set_ylim([-8,8])
ax.set_title(aa+'; '+str(decoder_results[session_id]['results'][aa]['n_units'])+' units')

In [ ]:
fig,ax=plt.subplots(1,1)
ax.plot(decoder_results[session_id]['middle_4_blocks'],decision_function_shifts.T,color='grey',alpha=0.1)
ax.plot(decoder_results[session_id]['middle_4_blocks'],np.median(decision_function_shifts,axis=0),color='k',linewidth=2,label='median null decoding')
ax.plot(decoder_results[session_id]['middle_4_blocks'],decision_function_shifts[shifts[half_shift_inds]==1,:].T,'r.',linewidth=2,alpha=0.5,label='true decoding')
ax.plot(decoder_results[session_id]['middle_4_blocks'],true_label*15-7.5,'b--',linewidth=2,alpha=0.5,label='true block label')
ax.set_xlabel('trial')
ax.set_ylabel('decision function')
ax.set_ylim([-8,8])
ax.set_title(aa+' n='+str(decoder_results[session_id]['results'][aa]['n_units'])+' units')
# ax.legend()
# ax.plot(decoder_results[session_id]['middle_4_blocks'],decision_function_shifts[shifts[half_shift_inds[0]],:].T,color='b',linewidth=1)
# ax.plot(decoder_results[session_id]['middle_4_blocks'],decision_function_shifts[shifts[half_shift_inds[-1]],:].T,color='r',linewidth=1)

In [ ]:
decision_function_shifts[shifts[half_shift_inds]==1,:].flatten()-np.median(decision_function_shifts,axis=0)

In [ ]:
fig,ax=plt.subplots(1,1)
ax.axhline(0,color='k')
ax.plot(decoder_results[session_id]['middle_4_blocks'],decision_function_shifts[shifts[half_shift_inds]==1,:].flatten()-np.median(decision_function_shifts,axis=0),'r.',alpha=0.5,label='corrected decoding')
ax.plot(decoder_results[session_id]['middle_4_blocks'],true_label*10-5,'b--',linewidth=2,alpha=0.5,label='true block label')
ax.set_xlabel('trial')
ax.set_ylabel('corrected decision function')
ax.set_title(aa+' n='+str(decoder_results[session_id]['results'][aa]['n_units'])+' units')


In [ ]:
adj_dec_func=decision_function_shifts[shifts[half_shift_inds]==1,:].flatten()-np.median(decision_function_shifts,axis=0)
adj_dec_func=adj_dec_func/np.std(adj_dec_func)
np.mean(adj_dec_func[true_label==1])
np.mean(adj_dec_func[true_label==0])

In [ ]:
# decoder_results[session_id]['middle_4_blocks']
# correct_target_trials

In [ ]:
trials=pd.read_parquet(
            npc_lims.get_cache_path('trials',session_id,version='any')
        )

trials_middle=trials.iloc[decoder_results[session_id]['middle_4_blocks'],:]
trials_middle.reset_index(inplace=True)
trials_middle.columns

In [ ]:
correct_target_trials=trials_middle.query('is_correct==True and is_target==True and is_response==True').index.values
cr_target_trials=trials_middle.query('is_correct==True and is_target==True and is_response==False').index.values
incorrect_target_trials=trials_middle.query('is_correct==False and is_target==True').index.values

fig,ax=plt.subplots(1,1)
ax.axhline(0,color='k')
ax.plot(decoder_results[session_id]['middle_4_blocks'],true_label*12-6,'k--',linewidth=2,alpha=0.3)
ax.plot(decoder_results[session_id]['middle_4_blocks'][correct_target_trials],adj_dec_func[correct_target_trials],'g.',alpha=0.5,label='hit')
ax.plot(decoder_results[session_id]['middle_4_blocks'][cr_target_trials],adj_dec_func[cr_target_trials],'b.',alpha=0.5,label='cr')
ax.plot(decoder_results[session_id]['middle_4_blocks'][incorrect_target_trials],adj_dec_func[incorrect_target_trials],'r.',alpha=0.5,label='fa')
ax.set_xlabel('trial')
ax.set_ylabel('corrected decision function')
ax.legend()

In [ ]:
vis_HIT_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==True and is_target==True and is_vis_context==True and is_response==True)').index.values])
aud_HIT_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==True and is_target==True and is_vis_context==False and is_response==True)').index.values])
vis_CR_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==True and is_target==True and is_vis_context==True and is_response==False)').index.values])
aud_CR_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==True and is_target==True and is_vis_context==False and is_response==False)').index.values])
vis_FA_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==False and is_target==True and is_vis_context==True and is_response==True)').index.values])
aud_FA_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==False and is_target==True and is_vis_context==False and is_response==True)').index.values])

fig,ax=plt.subplots(1,1)

barlist=ax.bar(['vis_HIT','vis_CR','vis_FA','aud_HIT','aud_CR','aud_FA'],[vis_HIT_mean,vis_CR_mean,vis_FA_mean,aud_HIT_mean,aud_CR_mean,aud_FA_mean])
ax.set_ylabel('corrected decision function')
barlist[0].set_color('tab:green')
barlist[1].set_color('tab:blue')
barlist[2].set_color('tab:red')
barlist[3].set_color('tab:green')
barlist[4].set_color('tab:blue')
barlist[5].set_color('tab:red')
ax.set_title(aa+' n='+str(decoder_results[session_id]['results'][aa]['n_units'])+' units')
ax.set_ylim([-2,2])

In [ ]:
np.mean(adj_dec_func[trials_middle.query('is_correct==True and is_target==True and is_vis_context==True').index.values])

In [ ]:
np.mean(adj_dec_func[trials_middle.query('is_correct==True and is_target==True and is_aud_context==True').index.values])

In [ ]:
vis_HIT_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==True and is_target==True and is_vis_context==True and is_response==True)').index.values])
aud_HIT_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==True and is_target==True and is_vis_context==False and is_response==True)').index.values])
print(vis_HIT_mean,aud_HIT_mean)

In [ ]:
vis_CR_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==True and is_target==True and is_vis_context==True and is_response==False)').index.values])
aud_CR_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==True and is_target==True and is_vis_context==False and is_response==False)').index.values])
print(vis_CR_mean,aud_CR_mean)

In [ ]:
vis_FA_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==False and is_target==True and is_vis_context==True and is_response==True)').index.values])
aud_FA_mean=np.mean(adj_dec_func[trials_middle.query('(is_correct==False and is_target==True and is_vis_context==False and is_response==True)').index.values])
print(vis_FA_mean,aud_FA_mean)

In [ ]:
session_id=list(decoder_results.keys())[15]

shifts=decoder_results[session_id]['shifts']
areas=decoder_results[session_id]['areas']

half_neg_shift=np.round(shifts.min()/2)
half_pos_shift=np.round(shifts.max()/2)
# half_shifts=np.arange(-half_neg_shift,half_pos_shift+1)
half_neg_shift_ind=np.where(shifts==half_neg_shift)[0][0]
half_pos_shift_ind=np.where(shifts==half_pos_shift)[0][0]
half_shift_inds=np.arange(half_neg_shift_ind,half_pos_shift_ind+1)

bal_acc={}
for aa in areas:
    if aa in decoder_results[session_id]['results']:
        bal_acc[aa]=[]
        for sh in half_shift_inds:
            bal_acc[aa].append(decoder_results[session_id]['results'][aa]['shift'][sh]['balanced_accuracy'])
        bal_acc[aa]=np.array(bal_acc[aa])

In [ ]:
test=pd.read_pickle(r"\\allen\programs\mindscope\workgroups\dynamicrouting\Ethan\new_annotations\single unit metrics\test linear shift\690706_2023-11-27_0_stim_context_modulation.pkl")
test

In [ ]:
test.columns.values

In [ ]:
np.mean((test['linear_shift_baseline_context_p_value_higher'].values<0.05) | (test['linear_shift_baseline_context_p_value_lower'].values<0.05))

In [ ]:
np.mean(test['baseline_context_modulation_p_value'].values)

In [ ]:
vals=np.hstack(np.abs(test['linear_shift_baseline_context_true_value'].values-test['linear_shift_baseline_context_null_median'].values))
fig,ax=plt.subplots(1,1)
ax.hist(vals,bins=20)

In [ ]:
#open pickle file
path=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\new_annotations\test\690706_2023-11-30_0_decoding_test_LDA.pkl"
with open(path,'rb') as f:
    test=pickle.load(f)

In [ ]:
test['690706_2023-11-30'].keys()

In [ ]:
shifts=test['690706_2023-11-30']['shifts']
shifts

In [ ]:
test['690706_2023-11-30']['results']['VISp']['shift'][np.where(shifts==1)[0][0]]['balanced_accuracy']

In [ ]:
non_annotated_sessions = tuple(s for s in npc_lims.get_session_info(is_ephys=True, is_uploaded=True, is_annotated=False))
non_annotated_sessions

In [ ]:
decoder_results[session_id]['results'].keys()

In [ ]:
decoder_results[session_id]['results']['MOs_probeA']['unit_ids']

In [ ]:
aligned_shift=np.where(decoder_results[session_id]['shifts']==1)[0][0]
decoder_results[session_id]['results']['MOs_probeA']['shift'][aligned_shift]['coefs']